In [1]:
import sys

sys.path.append("..")
from core.DataLoader import (
    DataPreprocessor,
    DataConfig,
    LoadConfig,
    get_load_config_from_yaml,
    combine_train_datasets
)
import numpy as np
from importlib import reload
import matplotlib.pyplot as plt
import yaml
import core.assignment_models as Models
import core
import keras
import tensorflow as tf


PLOTS_DIR = f"plots/mixed_model/"
MODEL_DIR = f"models/mixed_model"
#MODEL_DIR = f"../models/FeatureConcatTransformer_HLF_d256_l10_h8"
CONFIG_PATH = "../config/workspace_config.yaml"

import os

if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR, exist_ok=True)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR, exist_ok=True)

load_config = get_load_config_from_yaml(CONFIG_PATH)

DataProcessorNominal = DataPreprocessor(load_config)
DataProcessorToponium = DataPreprocessor(load_config)

with open(CONFIG_PATH, "r") as file:
    data_configs = yaml.safe_load(file)

plt.rcParams.update({"font.size": 14})

data_config = DataProcessorNominal.load_from_npz(data_configs["data_path"]["nominal"], max_events=2000000)
DataProcessorToponium.load_from_npz(data_configs["data_path"]["toponium"])

X_nominal, y_nominal = DataProcessorNominal.get_data()
X_toponium, y_toponium = DataProcessorToponium.get_data()
X_train, y_train = combine_train_datasets(
    [X_nominal, X_toponium],
    [y_nominal, y_toponium],
    weights=[0.5, 0.5],
)

2025-12-01 11:42:58.804754: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764585778.826761 3096413 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764585778.834115 3096413 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764585778.851833 3096413 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764585778.851851 3096413 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764585778.851854 3096413 computation_placer.cc:177] computation placer alr

In [ ]:
reload(Models)
reload(core)
TransformerMatcher = Models.CrossAttentionModel(data_config, name=r"Transformer + $\nu^2$-Flows")

#TransformerMatcher.load_model(f"{MODEL_DIR}/model.keras")
TransformerMatcher.build_model(
    hidden_dim=64,
    num_layers=8,
    num_heads=8,
    dropout_rate=0.1,
)

TransformerMatcher.adapt_normalization_layers(X_train)

TransformerMatcher.compile_model(
    loss={
        "assignment": core.utils.AssignmentLoss(lambda_excl=0.0),
    },
    optimizer=keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-4),
    metrics={
        "assignment": [core.utils.AssignmentAccuracy()],
    },
)
TransformerMatcher.model.summary()

I0000 00:00:1764585908.318199 3096413 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15511 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:65:00.0, compute capability: 6.0


Building model without regression output.


Model: "CrossAttentionModel"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ met_inputs          │ (None, 1, 2)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_input_transform │ (None, 1, 2)      │          0 │ met_inputs[0][0]  │
│ (InputMetPhiLayer)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_inputs          │ (None, 6, 5)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_input_normaliz… │ (None, 1, 2)      │          5 │ met_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_input_transform │ (None, 6, 5)      │          0 │ jet_inputs[0][0]  │
│ (InputPtEtaPhiELay… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 2)         │          0 │ met_input_normal… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_input_normaliz… │ (None, 6, 5)      │         11 │ jet_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector       │ (None, 6, 2)      │          0 │ flatten[0][0]     │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 6, 7)      │          0 │ jet_input_normal… │
│ (Concatenate)       │                   │            │ repeat_vector[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_embedding (MLP) │ (None, 6, 128)    │      7,328 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_mask            │ (None, 6, 1)      │          0 │ jet_inputs[0][0]  │
│ (GenerateMask)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_inputs          │ (None, 2, 4)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ self_attention_blo… │ (None, 6, 128)    │    132,480 │ jet_embedding[0]… │
│ (SelfAttentionBloc… │                   │            │ jet_mask[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_input_transform │ (None, 2, 4)      │          0 │ lep_inputs[0][0]  │
│ (InputPtEtaPhiELay… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ self_attention_blo… │ (None, 6, 128)    │    132,480 │ self_attention_b… │
│ (SelfAttentionBloc… │                   │            │ jet_mask[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_input_normaliz… │ (None, 2, 4)      │          9 │ lep_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector_1     │ (None, 2, 2)      │          0 │ flatten[0][0]     │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 1,639,095 (6.25 MB)

 Trainable params: 1,639,070 (6.25 MB)

 Non-trainable params: 25 (112.00 B)

In [3]:
TransformerMatcher.train_model(
    epochs=100,
    X_train=X_train,
    y_train=y_train,
    sample_weights=core.utils.compute_sample_weights(X_train, y_train),
    batch_size=1024,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.5,
            patience=5,
            verbose=1,
            mode="min",
            min_lr=1e-6,
        ),
    ],
)

Epoch 1/100


I0000 00:00:1764585946.917150 3099620 service.cc:152] XLA service 0x7fa60c01c150 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1764585946.917176 3099620 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2025-12-01 11:45:48.200069: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-12-01 11:45:49.799009: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator CrossAttentionModel_1/met_input_transform_1/assert_equal_1/Assert/Assert
I0000 00:00:1764585953.322802 3099620 cuda_dnn.cc:529] Loaded cuDNN version 91500
2025-12-01 11:46:22.503574: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_fusion_8', 48 bytes spill stores, 48 bytes spill loads
ptxas war

   1/1377 ━━━━━━━━━━━━━━━━━━━━ 25:43:14 67s/step - accuracy: 0.0859 - loss: 0.2645

I0000 00:00:1764585983.117779 3099620 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1376/1377 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - accuracy: 0.2237 - loss: 0.1436

2025-12-01 11:49:00.836264: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator CrossAttentionModel_1/met_input_transform_1/assert_equal_1/Assert/Assert
2025-12-01 11:49:29.351491: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'input_multiply_reduce_select_fusion', 20 bytes spill stores, 20 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_multiply_reduce_select_fusion_1', 16 bytes spill stores, 16 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_fusion_15', 64 bytes spill stores, 64 bytes spill loads



1377/1377 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - accuracy: 0.2238 - loss: 0.1436

2025-12-01 11:49:32.323495: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator CrossAttentionModel_1/met_input_transform_1/assert_equal_1/Assert/Assert
2025-12-01 11:49:41.758986: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator CrossAttentionModel_1/met_input_transform_1/assert_equal_1/Assert/Assert


1377/1377 ━━━━━━━━━━━━━━━━━━━━ 268s 146ms/step - accuracy: 0.2238 - loss: 0.1436 - val_accuracy: 0.5872 - val_loss: 0.0999 - learning_rate: 1.0000e-04
Epoch 2/100
1377/1377 ━━━━━━━━━━━━━━━━━━━━ 162s 118ms/step - accuracy: 0.5290 - loss: 0.1077 - val_accuracy: 0.6571 - val_loss: 0.0906 - learning_rate: 1.0000e-04
Epoch 3/100
1377/1377 ━━━━━━━━━━━━━━━━━━━━ 162s 117ms/step - accuracy: 0.6049 - loss: 0.0979 - val_accuracy: 0.6895 - val_loss: 0.0855 - learning_rate: 1.0000e-04
Epoch 4/100
1377/1377 ━━━━━━━━━━━━━━━━━━━━ 162s 118ms/step - accuracy: 0.6365 - loss: 0.0932 - val_accuracy: 0.6965 - val_loss: 0.0841 - learning_rate: 1.0000e-04
Epoch 5/100
1377/1377 ━━━━━━━━━━━━━━━━━━━━ 162s 117ms/step - accuracy: 0.6557 - loss: 0.0902 - val_accuracy: 0.6952 - val_loss: 0.0849 - learning_rate: 1.0000e-04
Epoch 6/100
1377/1377 ━━━━━━━━━━━━━━━━━━━━ 162s 117ms/step - accuracy: 0.6690 - loss: 0.0880 - val_accuracy: 0.6972 - val_loss: 0.0850 - learning_rate: 1.0000e-04
Epoch 7/100
1377/1377 ━━━━━━━━━━━━


KeyboardInterrupt



In [4]:
MODEL_DIR = f"models/mixed_cross_attention_model/"
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR, exist_ok=True)
TransformerMatcher.save_model(f"{MODEL_DIR}/model.keras")

Model saved to models/mixed_cross_attention_model//model.keras
